## Movie distribution


The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Implementation

In [10]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    def start_check(graph):
        demand = 0
        for state in graph.nodes():
            demand += graph.node[state]['demand']
        if demand != 0:
            raise nx.NetworkXUnfeasible("An error is thrown if there is no flow satisfying the demands.")
        return
    
    def build_super(G):
        G.add_node('S')
        G.add_node('T')
        for state in G.nodes():
            if state != 'S'and state != 'T':
                if G.node[state]['demand'] < 0:
                    G.add_edge('S', state)
                    G.edge['S'][state]['capacity'] = -G.node[state]['demand']
                if G.node[state]['demand'] > 0:
                    G.add_edge(state, 'T')
                    G.edge[state]['T']['capacity'] = G.node[state]['demand']
        return G
    
    def reset(G):
        keys, values = [], []
        for v in G.nodes():
            keys.append(v)
            res = dict()
            for (v1, v2) in G.edges():
                if v1 == v:
                    res[v2] = 0
            values.append(res)
        flow = dict()
        for i, j in enumerate(keys):
            flow[j] = values[i]
        return flow

    def residual(G, flow):
        G_r = nx.DiGraph()
        for (s1, s2) in G.edges():
            if flow[s1][s2] > 0:
                G_r.add_edge(s2, s1)
                G_r[s2][s1]['capacity'] = flow[s1][s2]
            if flow[s1][s2] < G[s1][s2]['capacity']:
                G_r.add_edge(s1, s2)
                G_r[s1][s2]['capacity'] = G[s1][s2]['capacity'] - flow[s1][s2]
        return G_r

    def bfs(G, node):
        queue,discovered,prev = [node],[node],{}
        while len(queue) > 0:
            u = queue[0]
            queue = queue[1:]
            for v in G.neighbors(u):
                if v not in discovered:
                    prev[v] = u
                    discovered.append(v)
                    queue.append(v)
        return [prev, discovered]

    def Path_Helper(G, s, t):
        prev, discovered = bfs(G, s)
        if t in discovered:
            path = [t]
            key = t
            while key != None:          
                if key in prev.keys():
                    path.append(prev[key])
                    key = prev[key]
                else: 
                    key = None
            return path[::-1]
        else:
            return None
            
    def Augmenting_Path(G, path, G_r, flow):
        capacities = []
        for i in range(len(path)-1):
            s1 = path[i]
            s2 = path[i+1]
            capacities.append(G_r.edge[s1][s2]['capacity'])
        addition = min(capacities)
        for i in range(len(path)-1):
            s1 = path[i]
            s2 = path[i+1]
            if (s1, s2) in G.edges():
                flow[s1][s2] = flow[s1][s2] + addition
            else:
                flow[s2][s1] = flow[s2][s1] - addition
        return flow
    
    def delete_super(flow):     
        for key in flow.keys():
            if 'S' in flow[key].keys():
                del flow[key]['S']
            if 'T' in flow[key].keys():
                del flow[key]['T']
        del flow['S']
        del flow['T']
        return flow
    
    def final_check(G):
        max_flow,demand = 0, 0
        for (s1, s2) in G.edges():
            if s1 == 'S':
                max_flow += flow['S'][s2]  
        for state in G.nodes():
            if state != 'S' and state != 'T':
                if G.node[state]['demand'] > 0:
                    demand += G.node[state]['demand']      
        if max_flow != demand:
            raise nx.NetworkXUnfeasible("An error is thrown if there is no flow satisfying the demands.")
        return  
        
    start_check(graph)
    G = build_super(graph.copy())
    flow = reset(G)
    G_r = residual(G, flow)
    path = Path_Helper(G_r, 'S', 'T') 
    while path:
        flow = Augmenting_Path(G, path, G_r, flow)
        G_r = residual(G, flow)
        path = Path_Helper(G_r, 'S', 'T')
    final_check(G)
    
    return delete_super(flow)


Implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [11]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    in_flow, out_flow, net_flow = {},{},{}
    for i in flow.keys():
        in_flow[i],out_flow[i],net_flow[i] = 0,0,0
    for v1 in flow.keys():
        out_flow[v1] = sum(flow[v1].values())
        for v2 in flow[v1].keys():
            in_flow[v2] += flow[v1][v2]
            
    for i in flow.keys():
        net_flow[i] = in_flow[i] - out_flow[i]
    return net_flow

The following code performs a sanity check my function (but does not completely confirm correctness).

In [12]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
